### What are logs?

Logging is a way of tracking events that happen when some software runs.

We add logging calls to our code to indicate that certain events have occured.  The log messages can include the values of important variables at the time.

Each log message has a *level* or *severity* which tells us something about the importance of the event.

### Structured vs Human readable logs

The focus here is on human readable logs, but some logs (e.g. webserver access logs,  error logs, avro/parquet event logs) have a structured format and are designed to be processed  by another program.

### Where do logs go?

Logging best practices have changed.

It used to be that in production, applications were responsible for managing their logs -- and logs would go to specific log files (sometimes more than one), either directly or through a system service called syslogd.

Now, the new model (https://12factor.net/logs) is that applications aren't responsible for routing their logs, and their logs are simply printed out -- they go to a single stream (stdout / the console).  And then a system down the road is responsible for organizing them and making them accessible.

But sometimes writing logs to disk is still useful for development (or services running on permanent servers). 

### Logging in python

Logging is done through a `logger` object with methods that publish logs at specific "log levels":
```python
logger.debug(f"Polishing rocket id={id} in preparation for launch.")
logger.info(f"Launching rocket id={id} targetted at location_id={location_id}")
logger.warning(f"Rocket id={id} is making alarming noises.")
try:
    rocket.launch()
catch LaunchException, e:
    logger.exception(f"Rocket id={id} has failed to launch.")
logger.critical(f"Rocker id={id} has exploded on the launchpad.")
```

## Log levels

| What you're doing:                                                                                                                   | Log Level         |
|:-------------------------------------------------------------------------------------------------------------------------------------|:------------------|
| Reporting normal events that happen during the application                                                                           | debug or info     |
| Issuing a warning that something is unexpected that might indicate a problem                                                         | warning           |
| Report that there's been an exception but program is still running | exception (error) |
| Report that the system has crashed or it might be unable to continue                                                                 | critical          |

| Severity Level | When it's used                                                                                                                                                                                               |
|:--|:--|
| debug          | Detailed information, typically of interest only when diagnosing problems.                                                                                                                                   |
| info           | Confirmation that things are working as expected.                                                                                                                                                            |
| warning        | An indication that something unexpected happened, or indicative of some problem in the near future  (e.g. ‘disk space low’). The software is still working as expected.  It's worth a day time notification. |
| error      | Due to a more serious problem, the software has not been able to perform some function.   It's worth a daytime or on-call notification, depending on the system.                                             |
| critical       | The system has failed.  It's worth the highest priority alert the system has.                                                                                                                                |

### What makes a good log?

A good log is useful for its intended purpose, which is a broad definition.  But generally, more 
*context* makes a log message more useful.

Instead of just saying something failed, explain what it was doing when it failed, what it was trying to do, and why.

(example cribbed from atlassian)
```python
// bad
logger.exception("Unable to save page.")
// better
logger.exception(f"Unable to save page {page_id}.")
// best
logger.exception(f"Unable to save {page} in updateMode={is_automatic_save} upon moving it " \
                 f"from space {oldSpace} to space {newSpace}.")
```

### Getting a python logger

The first step is getting a python logger instance.

Every python logger has a name, which is usually also the name of a python module or class.  That name can be included in every log message, which helps provide context for the log message.

In [1]:
# getting a logger through the core project helper function
from core.logging import get_logger

In [2]:
# manually providing the logger name
logger = get_logger("core.notebooks")
logger.warning("Logging a debug message")

2019-02-05 18:31:29,954 - core.notebooks - WARNING - Logging a debug message


In [3]:
# get_logger is our version of `logging.getLogger` in the core project that ensures
# our logging is set up.  This is what it looks like to use python logging directly:
import logging
logger2 = logging.getLogger("core.notebooks.logging")

In [4]:
logger.debug("hi")

2019-02-05 16:23:11,929 - core.notebooks.logging - DEBUG - hi


Inside of a python module, the python idiom is to use `__name__`:

```python
from core.logging import get_logger
logger = get_logger(__name__)
```

and if it's a class, use `get_logger(__name__)` in the init:

```python
class Foo:
    def __init__(self, id):
        self.logger = get_logger(__name__)
        self.id = id
        
    def launch_missiles(self):
        self.logger.info(f"Missile with id={self.id} is now being launched.")
```

but I've added a LoggerMixin class, so you can just write:

```python
class Foo(LoggerMixin):
    def launch_missiles(self):
        self.logger.info(f"Missile with id={self.id} is now being launched.")
```

and the logger name for the mixin includes the class name.

When we generate messages of severity `error`, we usually want to use the `exception` method that includes the exception and Traceback information.

In [5]:
def throw_exception(logger):
    try:
        "hello" + 3
    except TypeError as err:
        logger.exception("Something has clearly gone wrong")
        pass  # we can continue on, having logged the exception

throw_exception(logger)


2019-02-05 16:23:12,373 - core.notebooks.logging - ERROR - Something has clearly gone wrong
Traceback (most recent call last):
  File "<ipython-input-5-d5f53107b3b3>", line 3, in throw_exception
    "hello" + 3
TypeError: must be str, not int


### Configuring python logging

The most compelling reason to learn how to configure the logger is that accessing the logs of the libraries we're using can be very helpful when trying to debug behavior.

In [6]:
import requests
requests.get("http://google.com")

<Response [200]>

In [7]:
logging.getLogger("urllib3").setLevel(logging.DEBUG)
requests.get("http://google.com")
logging.getLogger("urllib3").setLevel(logging.INFO)

2019-02-05 16:23:13,079 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): google.com:80
2019-02-05 16:23:13,160 - urllib3.connectionpool - DEBUG - http://google.com:80 "GET / HTTP/1.1" 301 219
2019-02-05 16:23:13,164 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): www.google.com:80
2019-02-05 16:23:13,345 - urllib3.connectionpool - DEBUG - http://www.google.com:80 "GET / HTTP/1.1" 200 5520


In [8]:
import pytest
from core.constants import DEV_BUCKET
from core.helpers.configuration_mocker import ConfigurationMocker as CMock
import core.models.configuration as C
import core.contract as contract
from core.transforms.shared.raw.extract import ExtractTransform
import boto3
import moto
import time
import tempfile
import os

In [9]:
@moto.mock_s3
def s3_setup():
    client = boto3.client('s3')
    n_time = time.time()
    time_delta = 10000000
    output_contract = contract.Contract(branch='master', parent='bluth', child='cornballer',state='raw',env='dev')
    client.create_bucket(Bucket= DEV_BUCKET)

    t_file_old = tempfile.NamedTemporaryFile()
    t_file_old.write(b'Gobias some old coffee!')
    file_name_old = os.path.split(t_file_old.name)[1]
    output_contract.set_file_name(file_name_old)
    client.upload_file(Bucket=DEV_BUCKET, Filename= t_file_old.name, Key= output_contract.get_key(), ExtraArgs={"Metadata":{"source_modified_time": str(n_time - time_delta)}})
    
    t_file_new = tempfile.NamedTemporaryFile()
    t_file_new.write(b'Gobias some new coffee!')
    file_name_new = os.path.split(t_file_new.name)[1]
    output_contract.set_file_name(file_name_new)
    client.upload_file(Bucket=DEV_BUCKET, Filename= t_file_new.name, Key= output_contract.get_key(), ExtraArgs={"Metadata":{"source_modified_time": str(n_time + time_delta)}})
    
    return (t_file_old, t_file_new, output_contract, time_delta)


In [10]:
@moto.mock_s3
def test_push_to_s3_updated_file():
    params = s3_setup()
    extract = ExtractTransform()
    client = boto3.client('s3')
    extract.push_to_s3(tmp_dir=os.path.dirname(params[0].name),
                       output_contract=params[2])
    params[2].set_file_name(os.path.split(params[0].name)[1])
    s3_time = float(client.head_object(Bucket=DEV_BUCKET, Key=params[2].get_key())['Metadata']['source_modified_time'])
    
    assert os.stat(params[0].name).st_mtime == s3_time             


In [11]:
test_push_to_s3_updated_file()

2019-02-05 16:23:15,733 - botocore.credentials - INFO - Found credentials in environment variables.
2019-02-05 16:23:15,838 - core.contract.Contract - INFO - Publishing a local file at /tmp/tmpzc07pig0 to s3 location s3://ichain-dev/master/bluth/cornballer/raw/tmpzc07pig0.


That's usually good enough.  Let's see everything that's being logged, by all modules.

Below, we'll programmatically grab the *root* logger and configure it to show us all messages at the DEBUG level or higher, which will show all logged messages.

In [12]:
import logging
root_logger = logging.getLogger('')
root_logger.setLevel(logging.DEBUG)

In [13]:
test_push_to_s3_updated_file()

2019-02-05 16:23:15,903 - botocore.hooks - DEBUG - Event choose-service-name: calling handler <function handle_service_name_alias at 0x7fd90895c488>
2019-02-05 16:23:15,910 - botocore.hooks - DEBUG - Event creating-client-class.s3: calling handler <function add_generate_presigned_post at 0x7fd90899b048>
2019-02-05 16:23:15,912 - botocore.hooks - DEBUG - Event creating-client-class.s3: calling handler <function lazy_call.<locals>._handler at 0x7fd8f3cd7f28>
2019-02-05 16:23:15,914 - botocore.hooks - DEBUG - Event creating-client-class.s3: calling handler <function add_generate_presigned_url at 0x7fd908994d90>
2019-02-05 16:23:15,916 - botocore.args - DEBUG - The s3 config key is not a dictionary type, ignoring its value of: None
2019-02-05 16:23:15,920 - botocore.endpoint - DEBUG - Setting s3 timeout as (60, 60)
2019-02-05 16:23:15,923 - botocore.client - DEBUG - Registering retry handlers for service: s3
2019-02-05 16:23:15,927 - botocore.client - DEBUG - Defaulting to S3 virtual host 

### logger configuration

The internals of python logging are surprisingly involved, but it makes sense once you're used to it.  It's powerful, but it's built for a time when applications were expected to manage logs -- modern practice ( https://12factor.net/logs ) removes the responsibility of handling logs from applications, as they are simply supposed to print their logs to stdout.

First, let's take a glance at a configuration file and then we'll go through the core concepts that will help make sense of it.

https://github.com/IntegriChain1/core/blob/f8afa1b703ecec717acbfde83b695f97344c05b2/logging.yaml

Here are some key concepts.

There are loggers, handlers, and formatters.

The loggers are the main interface we use for creating messages, and each one has a name.  Each logger has a "log level" which indicates what level a log message has to be in order to not be ignored.  For example, if we set "core.contract" to INFO level, then a "core.contract" debug message will get ignored.

Every logger is part of a hierarchy, with the "root logger" as the parent of them all.  If the name of a logger is formed like a python package, e.g. `core.contract`, then the subpackage is a child of the parent package.  So the hierarchy of `core.contract` would be:

**Root logger** ← **"core" logger** ← **"core.contract" logger**

Each logger can have handlers, which actually do something -- like write the log to the console/stdout or write to a file.  There are lots of python logging handlers that can write to all kinds of systems.

Formatters determine the content and structure of the log messages.

![alt](https://cdn-images-1.medium.com/max/1600/1*SqUfUKXaoJlbDHL6WErPmw.png)

By default, a logger will pass its messages (that aren't ignored) to its parents, but it can be configured to not "propogate" the message upwards.  Let's think about some scenarios.

1) We attach a handler to the root logger (configured at level INFO) that writes logs to a file.  A 'core.contract' message gets generated at WARN.  It gets passed up to the root logger, and then the root logger passes it to its handler, and the log gets written to the file.   

2) Now we set the log level of the "core.contract" logger to WARN, and generate a "core.contract" message at INFO level.  The message is ignored, and it's never passed up to the root logger.

3) We want to see all of the messages generated in our core project.  We add a handler directly to the "core" logger to print out the logs to the console.  We set the "core" logger to DEBUG.  We set the core logger to not propogate messages to the root logger.  Now, we generate a DEBUG message at "core.contract".  It propogates up to "core" and gets logged.  (If we didn't set it to not propogate up from core, messages would also get passed to the root logger and could get printed twice.)

Here's an exercise -- how do we suppress that botocore.credentials INFO message from getting printed all of the time?

In [14]:
root_logger.setLevel(logging.INFO)

session = boto3.session.Session()

sqs = session.client('sqs')
s3 = session.resource('s3')


2019-02-05 16:23:16,699 - botocore.credentials - INFO - Found credentials in environment variables.
